In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import os
from argparse import ArgumentParser, Namespace
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
import anndata as ad
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM
from gpu_spatial_graph_pipeline.data.datasets import DatasetHartmann
from gpu_spatial_graph_pipeline.data.anndata2data import ListAnnData2DataDefault
from gpu_spatial_graph_pipeline.data.utils import get_adjacency_from_adata, add_design_matrix

In [24]:
#Mibitof
dataset = DatasetHartmann(data_path='./example_data/hartmann/')
adatas = list(dataset.img_celldata.values())


Loading data from raw files
registering celldata


/home/sel/mambaforge/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates
Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.


In [25]:
adatas[32].uns

OverloadedDict, wrapping:
	OrderedDict([('metadata', {'lateral_resolution': 0.390625, 'fn': ['scMEP_MIBI_singlecell/scMEP_MIBI_singlecell.csv', 'scMEP_sample_description.xlsx'], 'image_col': 'point', 'pos_cols': ['center_colcoord', 'center_rowcoord'], 'cluster_col': 'Cluster', 'cluster_col_preprocessed': 'Cluster_preprocessed', 'patient_col': 'donor'}), ('img_keys', ['scMEP_point_1', 'scMEP_point_10', 'scMEP_point_11', 'scMEP_point_12', 'scMEP_point_13', 'scMEP_point_14', 'scMEP_point_15', 'scMEP_point_16', 'scMEP_point_17', 'scMEP_point_18', 'scMEP_point_19', 'scMEP_point_2', 'scMEP_point_20', 'scMEP_point_21', 'scMEP_point_22', 'scMEP_point_23', 'scMEP_point_24', 'scMEP_point_25', 'scMEP_point_26', 'scMEP_point_27', 'scMEP_point_28', 'scMEP_point_29', 'scMEP_point_3', 'scMEP_point_30', 'scMEP_point_31', 'scMEP_point_32', 'scMEP_point_33', 'scMEP_point_34', 'scMEP_point_35', 'scMEP_point_36', 'scMEP_point_37', 'scMEP_point_38', 'scMEP_point_39', 'scMEP_point_4', 'scMEP_point_40', 'scM

In [3]:
#Mibitof
dataset = DatasetHartmann(data_path='./example_data/hartmann/')
adatas = list(dataset.img_celldata.values())


fields = {
    'features':['obs/Cluster_preprocessed','obs/donor','obsm/design_matrix'],
    'labels':['X']
}



a2d = ListAnnData2DataDefault(
    fields=fields,
    preprocess=[lambda x,_: add_design_matrix(x,'obs/Cluster_preprocessed','obs/donor','design_matrix')]
)


Loading data from raw files
registering celldata


/home/sel/mambaforge/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates
Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.


In [26]:
a2d(adatas)

TypeError: Cannot interpret 'CategoricalDtype(categories=['CD4 T cells', 'CD68 Myeloid', 'CD8 T cells', 'Endothelial',
                  'Epithelial', 'Fibroblast', 'Other immune cells'],
, ordered=False)' as a data type

In [12]:
adatas[2].obs['point']

8059    scMEP_point_11
8060    scMEP_point_11
8061    scMEP_point_11
8062    scMEP_point_11
8063    scMEP_point_11
             ...      
9370    scMEP_point_11
9371    scMEP_point_11
9372    scMEP_point_11
9373    scMEP_point_11
9374    scMEP_point_11
Name: point, Length: 1316, dtype: category
Categories (1, object): ['scMEP_point_11']

In [56]:
adata.obsm["design_matrix"]

(63747, 76)

In [ ]:
dm = GraphAnnDataModule(adata=adata, adata2data_fn=a2d, num_workers = 12, batch_size=10,learning_type='graph')
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [ ]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu' if torch.torch.cuda.is_available() else 'cpu',
                                max_epochs=100,log_every_n_steps=10)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(model,datamodule=dm)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 1 dimension(s)

In [ ]:
trainer.test(model, datamodule=dm)

In [13]:
x = ad.concat(adata)

In [19]:
x.obs.donor

59191    90de
59192    90de
59193    90de
59194    90de
59195    90de
         ... 
18510    90de
18511    90de
18512    90de
18513    90de
18514    90de
Name: donor, Length: 63747, dtype: object

In [34]:
cats = x.obs.donor.unique()
for cat in cats:
    print(x[x.obs.donor == cat])

View of AnnData object with n_obs × n_vars = 18943 × 36
    obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
    obsm: 'spatial', 'node_types'
View of AnnData object with n_obs × n_vars = 22224 × 36
    obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
    obsm: 'spatial', 'node_types'
View of AnnData object with n_obs × n_vars = 5811 × 36
    obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
    obsm: 'spatial', 'node_types'
View of AnnData object with n_obs × n_vars = 16769 × 36
    obs: 'point', 'cell_id', 'cell_size', 'donor', 'Cluster', 'Cluster_preprocessed'
    obsm: 'spatial', 'node_types'


In [30]:
x.obs.donor.unique()

59191    90de
59192    90de
59193    90de
59194    90de
59195    90de
         ... 
18510    90de
18511    90de
18512    90de
18513    90de
18514    90de
Name: donor, Length: 63747, dtype: object